# Import Libraries and Datasets

In [ ]:
# INSTALLS

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 




E: Unable to locate package gdal-bin
E: Unable to locate package python-gdal
E: Unable to locate package python3-gdal



E: Unable to locate package python3-rtree
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-_9vbzoh4
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-_9vbzoh4
  Created wheel for geopandas: filename=geopandas-0.9.0+12.gafa49e0-py2.py3-none-any.whl size=995084 sha256=79d26784d574f2bba9eb60c6835189e92fbbcce01e3df1889739eb4c02918007
  Stored in directory: /tmp/pip-ephem-wheel-cache-b78bccny/wheels/cf/3e/0b/6475054094c2b1ea054158ac1fdcf749fb92f5b512377e4cf8
Successfully built geopandas
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
# IMPORTS

import pandas as pd
import numpy as np
import os
import geopandas as gpd
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
# MOUNT DRIVE

# !ls /datasets/datathondeepnoteaccess/Stanford_Datathon_103

# drive.mount('/content/gdrive/')
root_dir = '/datasets/datathondeepnoteaccess/'
base_dir = root_dir + 'Stanford_Datathon_103/'   

In [ ]:
# READ IN THE DATA (NOW USING DATA FROM GOOGLE DRIVE)

# Pre-processed THE data from here https://www.timeshighereducation.com/world-university-rankings-2021-table-information
# file = 'world_university_rankings_2016_to_2021_updated.csv'
file = base_dir + 'dataset/world_university_rankings_2016_to_2021_updated.csv'
dat_0 = pd.read_csv(file)

# CLEAN DATA

In [ ]:
# Rename columns
dat_1 = dat_0.rename(columns={'Year':'year_number',
                              'Rank':'rank_string',
                              'University':'uni',
                              'URL' :'url',
                              'Country':'country',
                              'Number of FTE Students':'FTEs',
                              'Number of Students per staff':'stdnts_stf_ratio',
                              'International Students':'intStdnts_ratio',
                              'Female Male Ratio':'fml_ml_ratio',
                              })

# Convert types (e.g., counts, ratios to floats and year to datetime... *rank is special case, see next)
dat_1['intStdnts_ratio'] = pd.to_numeric(dat_1.intStdnts_ratio, errors='coerce').fillna(0)
dat_1['year_dateTime'] = pd.to_datetime(dat_1.year_number, format='%Y')

# For rank given as a range, assign lowest value of range (e.g., for '251-300' assign '251')
dat_1['rank_number'] = dat_1.rank_string.str.extract('(\d+)').astype(int)

In [ ]:
dat_1.head()

/shared-libs/python3.7/py/lib/python3.7/site-packages/numpy/lib/histograms.py:822: RuntimeWarning: divide by zero encountered in double_scalars
  norm = n_equal_bins / _unsigned_subtract(last_edge, first_edge)
/shared-libs/python3.7/py/lib/python3.7/site-packages/numpy/lib/histograms.py:850: RuntimeWarning: invalid value encountered in multiply
  f_indices = _unsigned_subtract(tmp_a, first_edge) * norm


,rank_string,uni,url,country,FTEs,stdnts_stf_ratio,intStdnts_ratio,fml_ml_ratio,year_number,year_dateTime,rank_number
0,1,University of Oxford,https://www.timeshighereducation.com/world-uni...,United Kingdom,20774,11.1,0.41,46 : 54,2021,2021-01-01,1
1,2,Stanford University,https://www.timeshighereducation.com/world-uni...,United States,16223,7.4,0.23,44 : 56,2021,2021-01-01,2
2,3,Harvard University,https://www.timeshighereducation.com/world-uni...,United States,21261,9.3,0.25,49 : 51,2021,2021-01-01,3
3,4,California Institute of Technology,https://www.timeshighereducation.com/world-uni...,United States,2238,6.3,0.33,36 : 64,2021,2021-01-01,4
4,5,Massachusetts Institute of Technology,https://www.timeshighereducation.com/world-uni...,United States,11276,8.4,0.34,39 : 61,2021,2021-01-01,5


In [ ]:
# DW working on `fml_ml_ratio`

# annoyingly had NaN (float) values in the column
dat_1.fml_ml_ratio.fillna('', inplace=True)
# Taxe the first number (female) and make numeric
dat_1['fml_pct'] = pd.to_numeric(dat_1.fml_ml_ratio.apply(lambda x: x.split(' ')[0]))

# ORGANIZE DATA

In [ ]:
# Add column for region

dat_1.country.nunique()

# country had x severity of lockdown measures? etc.)

# Calculate the percent difference year over year


94

# INITIAL VIZ

## Rank over time (filtered to top 100 universities)

In [ ]:
# Timelines of individual schools (filter to top X for now) 
dat_vz = dat_1[['year_number','uni','rank_number']].query('rank_number <= 100')
# need to reshape from tall to wide
dat_vz = dat_vz.pivot(index='year_number', columns='uni', values='rank_number').reset_index()

# plot
fig = px.line(dat_vz, x='year_number', y=dat_vz.columns[:]) 
fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

## Percent female students, over time (filtered to top 300 unis)

In [ ]:
dat_vz = dat_1.query('rank_number <= 201')[['year_number','uni','fml_pct']]
dat_vz = dat_vz.pivot(index='year_number', columns='uni', values='fml_pct').reset_index() 
# plot
fig = px.line(dat_vz, x='year_number', y=dat_vz.columns[:]) 
fig.show()

## Percent International Students over Time

In [ ]:
# Timelines of individual schools (filter to top X for now) 
dat_vz = dat_1.query('rank_number <= 201')[['year_number','uni','intStdnts_ratio']]
dat_vz = dat_vz.pivot(index='year_number', columns='uni', values='intStdnts_ratio').reset_index()
# plot
fig = px.line(dat_vz, x='year_number', y=dat_vz.columns[:]) 
fig.show()

## Country-level mean percent international students, over time

In [ ]:
# Timelines of schools grouped into countries
dat_vz = dat_1.query('rank_number <= 101')[['year_number','uni','country','intStdnts_ratio']].groupby(['year_number','country']).mean().reset_index()
dat_vz = dat_vz.pivot(index='year_number', columns='country', values='intStdnts_ratio').reset_index()
# plot
fig = px.line(dat_vz, x='year_number', y=dat_vz.columns[:]) 
fig.show()

In [ ]:
dat_1.columns

Index(['rank_string', 'uni', 'url', 'country', 'FTEs', 'stdnts_stf_ratio',
       'intStdnts_ratio', 'fml_ml_ratio', 'year_number', 'year_dateTime',
       'rank_number', 'fml_pct'],
      dtype='object')

## Averages for universities, grouped by ranking bins (e.g., diffference between highest ranked and lowest ranked?)

In [ ]:
# scatter plot Ranking vs Int Percent
# do a regression controlling for country (i.e., location), is rank associated with percent international students in upper tier?
fig = px.scatter(dat_1.query('rank_number<=20'), x="rank_number", y="intStdnts_ratio", color="country", hover_data=['uni'],
opacity=0.65, trendline='ols')
fig.show() 

# do a regression controlling for country (i.e., location), is rank associated with percent international students in mid-upper tier?
fig = px.scatter(dat_1.query('rank_number>21 & rank_number<=100'), x="rank_number", y="intStdnts_ratio", color="country", hover_data=['uni'],
opacity=0.65, trendline='ols')
fig.show() 

In [ ]:
# Tiers of universities (i.e., top 5, top 25, top 50, ...): for each group, graph their percent of international students, over time
# Need to assign categories 'top 5' etc.  
dat_2 = dat_1.copy()
dat_2['tier'] = 'not_assigned'
conditions = [
    dat_2['rank_number']<= 10,
    (dat_2['rank_number'] > 10) & (dat_1['rank_number']<= 101),
    (dat_2['rank_number'] > 101) & (dat_1['rank_number']<= 201),
    (dat_2['rank_number'] > 201) & (dat_1['rank_number']< 1001),
    dat_2['rank_number'] == 1001
]

choices = ['A_top_10','B_top_100','C_top_250','D_top_1000','E_lowest_ranked']

dat_2['tier'] = np.select(conditions, choices, default='not_assigned')

# group on that to get a mean percent inter student for the group in a given year.
dat_vz = dat_2[['year_number','tier','intStdnts_ratio']].groupby(['year_number','tier'], as_index=False).mean()

# plot it
dat_vz = dat_vz.pivot(index='year_number', columns='tier', values='intStdnts_ratio').reset_index()
# plot
fig = px.line(dat_vz, x='year_number', y=dat_vz.columns[:], labels={'value':'Percent International Students','year_number':'Year'}, title='International students at different tiers of universities worldwide')
fig.show()

In [ ]:
# look at variability within tiers
 
dat_vz = dat_2[['year_number','tier','intStdnts_ratio']].groupby(['year_number','tier'], as_index=False).agg({'intStdnts_ratio':['mean','median','var']}).reset_index()
dat_vz

# look at group by size


# look at percentage change 


 result = df.groupby(['a'], as_index=False).agg(
                      {'c':['mean','std'],'b':'first', 'd':'first'})
 

# filter US only schools



year_number             tier intStdnts_ratio                 
                                           mean median       var
0         2016         A_top_10        0.309000  0.300  0.008188
1         2016        B_top_100        0.217447  0.190  0.013518
2         2016        C_top_250        0.162252  0.150  0.007768
3         2016       D_top_1000        0.098110  0.070  0.009596
4         2017         A_top_10        0.299091  0.270  0.009709
5         2017        B_top_100        0.216778  0.190  0.012660
6         2017        C_top_250        0.179351  0.150  0.010719
7         2017       D_top_1000        0.095386  0.060  0.010337
8         2018         A_top_10        0.312727  0.270  0.010262
9         2018        B_top_100        0.222967  0.200  0.013172
10        2018        C_top_250        0.190530  0.160  0.012422
11        2018       D_top_1000        0.102019  0.070  0.011305
12        2018  E_lowest_ranked        0.030588  0.015  0.002421
13        2019         A_top_10        0.314000  0.275  0.011649
14        2019        B_top_100        0.234835  0.200  0.014019
15        2019        C_top_250        0.203267  0.170  0.013083
16        2019       D_top_1000        0.109239  0.080  0.011761
17        2019  E_lowest_ranked        0.035814  0.020  0.002211
18        2020         A_top_10        0.318000  0.290  0.011596
19        2020        B_top_100        0.245934  0.210  0.015047
20        2020        C_top_250        0.204631  0.180  0.011620
21        2020       D_top_1000        0.114927  0.080  0.013122
22        2020  E_lowest_ranked        0.040606  0.020  0.002461
23        2021         A_top_10        0.285000  0.280  0.006450
24        2021        B_top_100        0.256413  0.220  0.017641
25        2021        C_top_250        0.215811  0.190  0.012457
26        2021       D_top_1000        0.121678  0.090  0.014319
27        2021  E_lowest_ranked        0.043410  0.030  0.002521

## DW Explorations

In [ ]:
list(dat_1.columns)

['rank_string',
 'uni',
 'url',
 'country',
 'FTEs',
 'stdnts_stf_ratio',
 'intStdnts_ratio',
 'fml_ml_ratio',
 'year_number',
 'year_dateTime',
 'rank_number',
 'fml_pct']

In [ ]:
# international students by year trend

# # plot
# fig = px.line(dat_1, x='year_number', y='intStdnts_ratio')
# fig.show()

fig = px.violin(dat_1, y="intStdnts_ratio", x="year_number", box=True, hover_data=dat_1.columns)
fig.show()

In [ ]:
# Regression line
fig = px.scatter(
    dat_1, x='year_number', y='intStdnts_ratio', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue'
)
fig.show()

In [ ]:
# all schools
dat_1.groupby('year_number')['intStdnts_ratio'].mean()

year_number
2016    0.126875
2017    0.121988
2018    0.119610
2019    0.116105
2020    0.113414
2021    0.113073
Name: intStdnts_ratio, dtype: float64

In [ ]:
# top 100
dat_1.loc[dat_1.rank_number<101,:].groupby('year_number')['intStdnts_ratio'].mean()

year_number
2016    0.224800
2017    0.226400
2018    0.232647
2019    0.243300
2020    0.253900
2021    0.261100
Name: intStdnts_ratio, dtype: float64

In [ ]:
# top 25
dat_1.loc[dat_1.rank_number<26,:].groupby('year_number')['intStdnts_ratio'].mean()

year_number
2016    0.279600
2017    0.279231
2018    0.293462
2019    0.281600
2020    0.273200
2021    0.280400
Name: intStdnts_ratio, dtype: float64

In [ ]:
pd.DataFrame(dat_1.groupby('country')['intStdnts_ratio'].mean()).sort_values(by='intStdnts_ratio', ascending=False)

,intStdnts_ratio
country,
Northern Cyprus,0.732000
Luxembourg,0.535000
Macao,0.461250
United Arab Emirates,0.459545
Qatar,0.408333
...,...
Nigeria,0.005625
Venezuela,0.002727
Tanzania,0.000000


In [ ]:
dat_1.loc[dat_1.country=='United States','intStdnts_ratio'].mean()

0.13178096212896623

### Canada Data

In [ ]:
# Checking out data re: Canada specifically
# Study permit holders by country of citizenship and year in which permit(s) became effective, January 2015 - January 2021

file = base_dir + 'other_data/IRCC_M_TRStudy_0001_E.xls'
df_c1 = pd.read_excel(file)

In [ ]:
df_c1.head()

,Country,Q1_2015,Y_2015,Q1_2016,Y_2016,Q1_2017,Y_2017,Q1_2018,Y_2018,Q1_2019,Y_2019,Q1_2020,Y_2020,Q1_2021
0,Afghanistan,15,90,25,105,20,85,10,75,25,95,15,85,10
1,Albania,15,110,25,155,20,175,25,240,65,350,60,235,25
2,Algeria,155,960,170,800,175,965,230,1450,425,2650,500,2065,230
3,Andorra,--,--,0,--,0,--,0,--,0,--,0,0,0
4,Angola,15,65,30,75,5,35,5,35,20,120,10,35,--


In [ ]:
# replace -- with 0
df_c1 = df_c1.replace('--',0)

In [ ]:
df_c1 = df_c1.sort_values(by="Y_2020", ascending=False).reset_index(drop=True)

In [ ]:
df_c1.head()

,Country,Q1_2015,Y_2015,Q1_2016,Y_2016,Q1_2017,Y_2017,Q1_2018,Y_2018,Q1_2019,Y_2019,Q1_2020,Y_2020,Q1_2021
0,India,2620,31920,4395,52630,6795,82945,13960,107065,15905,138780,16060,81235,7665
1,"China, People's Republic of",7925,65860,12300,76840,9585,82705,10710,85025,14430,84295,6985,58200,2500
2,"Korea, Republic of",3140,14735,4015,15935,3600,16700,3795,16865,4325,17000,3130,10980,825
3,Vietnam,425,2830,745,5320,1310,9865,1495,12360,2135,11625,1535,9405,555
4,France,1325,11860,1480,11825,1390,13225,1575,13420,2110,14545,1375,8005,2160


In [ ]:
test = df_c1.loc[:6,['Country', 'Q1_2015', 'Q1_2016', 'Q1_2017', 'Q1_2018', 'Q1_2019', 'Q1_2020', 'Q1_2021']]

In [ ]:
dat_vz = test.T

dat_vz.head()

#fig = px.line(dat_vz, x='year_number', y=dat_vz.columns[:]) 
#fig.show()

 

,0,1,2,3,4,5,6
Country,India,"China, People's Republic of","Korea, Republic of",Vietnam,France,Iran,Brazil
Q1_2015,2620,7925,3140,425,1325,315,1145
Q1_2016,4395,12300,4015,745,1480,480,1555
Q1_2017,6795,9585,3600,1310,1390,540,1780
Q1_2018,13960,10710,3795,1495,1575,710,2015


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  7 non-null      object
 1   Q1_2015  7 non-null      int64 
 2   Q1_2016  7 non-null      int64 
 3   Q1_2017  7 non-null      int64 
 4   Q1_2018  7 non-null      int64 
 5   Q1_2019  7 non-null      int64 
 6   Q1_2020  7 non-null      int64 
 7   Q1_2021  7 non-null      int64 
dtypes: int64(7), object(1)
memory usage: 576.0+ bytes


In [ ]:
list(df_c1.columns[df_c1.columns.str.contains('Q')])

['Q1_2015', 'Q1_2016', 'Q1_2017', 'Q1_2018', 'Q1_2019', 'Q1_2020', 'Q1_2021']

In [ ]:
# all schools
dat_1.groupby('year_number')['fml_pct'].mean()

year_number
2016    49.223262
2017    49.204104
2018    49.139446
2019    49.371048
2020    49.885049
2021    50.139023
Name: fml_pct, dtype: float64

In [ ]:
dat_1.loc[dat_1.rank_number<100,:].groupby('year_number')['fml_pct'].mean()

year_number
2016    50.033708
2017    49.258427
2018    49.136364
2019    49.932584
2020    50.000000
2021    49.931818
Name: fml_pct, dtype: float64

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5045dddd-59d5-47a3-a837-1b074a4d8efe' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>